# `AzureContentSafetyTextTool`

## Overview

### Integration details

>The `AzureContentSafetyTextTool` acts as a wrapper around the Azure AI Content Safety Service/API.
>The Tool will detect harmful content according to Azure's Content Safety Policy.

### Tool features

This integration allows for the detection of harmful or offensive content in text using Azure's Content Safety API. It supports four categories of harmful content: Sexual, Harm, Self-Harm, and Violence.

## Setup

This section provides details about how the Azure AI Content Safety integration works, including setup.

In [1]:
import os

from langchain import hub

We will use a prompt to instruct the model. LangChain prompts can be configured, but for simplicity, we will use a premade prompt from LangSmith. This requires an API key, which can be set up [here](https://www.langchain.com/langsmith) after registration.

In [2]:
LANGSMITH_KEY = os.environ["LANGSMITH_KEY"]
prompt = hub.pull("hwchase17/structured-chat-agent", api_key=LANGSMITH_KEY)

Now we can use the `AzureContentSafetyTextTool` combined with a model, using `create_structured_chat_agent`.

In [5]:
from langchain.agents import AgentExecutor, create_structured_chat_agent
from langchain_community.tools.azure_ai_services.content_safety import (
    AzureContentSafetyTextTool,
)
from langchain_openai import AzureChatOpenAI

### Credentials

Credentials can be set by being passed as parameters and should be stored locally as environment variables.

In [7]:
content_endpoint = os.environ["CONTENT_SAFETY_ENDPOINT"]
content_key = os.environ["CONTENT_SAFETY_KEY"]

## Instantiation

Credentials can be passed directly, but they can also be retrieved automatically by the constructor if environment variables named `CONTENT_SAFETY_ENDPOINT` and `CONTENT_SAFETY_KEY` are set.

In [8]:
cs = AzureContentSafetyTextTool(
    content_safety_key=content_key,
    content_safety_endpoint=content_endpoint,
)

In [9]:
tools = [cs]

model = AzureChatOpenAI(
    openai_api_version=os.environ["OPENAI_API_VERSION"],
    azure_deployment=os.environ["COMPLETIONS_MODEL"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
)

Creating an `AgentExecutor` chain allows a model to use tools to assist in it's response.

In [10]:
agent = create_structured_chat_agent(model, tools, prompt)

agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

## Invocation

Input must be in the form of a string (`str`).

In [11]:
input = "I hate you"

### [Invoke directly with args](/docs/concepts/#invoke-with-just-the-arguments)

Firstly, the tool can be used by directly passing input as an argument. However, this is discouraged as the tool is intended to be used in an executor chain.

In [12]:
cs.invoke({"query": input})

'Hate: 2\nSelfHarm: 0\nSexual: 0\nViolence: 0\n'

### [Invoke with ToolCall](/docs/concepts/#invoke-with-toolcall)

By using `.invoke`, the model can be told what to do and assess if using the tools it was given would assist in it's response. This is the intended use.

In [17]:
agent_executor.invoke(
    {"input": f"Can you check the following text for harmful content : {input}"}
)



> Entering new AgentExecutor chain...
{
  "action": "azure_content_safety_tool",
  "action_input": {
    "query": "I hate you"
  }
}Hate: 2
SelfHarm: 0
Sexual: 0
Violence: 0
{
  "action": "Final Answer",
  "action_input": "The text contains hate sentiment with a severity level of 2. There is no indication of self-harm, sexual content, or violence."
}

> Finished chain.


{'input': 'Can you check the following text for harmful content : I hate you',
 'output': 'The text contains hate sentiment with a severity level of 2. There is no indication of self-harm, sexual content, or violence.'}

## Chaining

When creating an `AgentExecutor` as described earlier, an execution chain is formed. The sequence of events will be printed using the given prompt, and actions will occur in a chain-like manner.

In [18]:
agent = create_structured_chat_agent(model, tools, prompt)

agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

## API reference

[Azure AI Content Safety Overview](https://learn.microsoft.com/azure/ai-services/content-safety/overview) | [Azure AI Content Safety Python SDK](https://learn.microsoft.com/python/api/overview/azure/ai-contentsafety-readme?view=azure-python)